# 使用 Semantic Kernel 與 OpenBnB MCP 伺服器整合

本筆記本展示如何使用 Semantic Kernel 與實際的 OpenBnB MCP 伺服器整合，透過 MCPStdioPlugin 搜尋真實的 Airbnb 住宿。對於 LLM 的存取，使用 Azure AI Foundry。要設定您的環境變數，您可以參考 [設定課程](/00-course-setup/README.md)。


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## 建立 MCP 插件連線

我們將使用 MCPStdioPlugin 連接到 [OpenBnB MCP 伺服器](https://github.com/openbnb-org/mcp-server-airbnb)。此伺服器透過 @openbnb/mcp-server-airbnb 套件提供 Airbnb 搜尋功能。


## 建立用戶端

在此範例中，我們將使用 Azure AI Foundry 來存取 LLM。請確保您的環境變數已正確設定。


## 環境設定

設定 Azure OpenAI 的相關配置。請確保已設定以下環境變數：
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## 理解 OpenBnB MCP 整合

此筆記本連接到 **真實的 OpenBnB MCP 伺服器**，提供實際的 Airbnb 搜尋功能。

### 運作方式：

1. **MCPStdioPlugin**：透過標準輸入/輸出與 MCP 伺服器進行通訊  
2. **真實的 NPM 套件**：使用 npx 下載並執行 `@openbnb/mcp-server-airbnb`  
3. **即時數據**：從 Airbnb 的 API 返回真實的房源數據  
4. **功能發現**：代理會自動發現 MCP 伺服器提供的可用功能  

### 可用功能：

OpenBnB MCP 伺服器通常提供以下功能：  
- **search_listings** - 根據地點和條件搜尋 Airbnb 房源  
- **get_listing_details** - 獲取特定房源的詳細資訊  
- **check_availability** - 檢查特定日期的可用性  
- **get_reviews** - 獲取房源的評論  
- **get_host_info** - 獲取房源主人的資訊  

### 先決條件：

- 系統上已安裝 **Node.js**  
- **網路連線** 以下載 MCP 伺服器套件  
- **NPX** 可用（Node.js 自帶）  

### 測試連線：

您可以手動測試 MCP 伺服器，執行以下命令：  
```bash
npx -y @openbnb/mcp-server-airbnb
```  

這將下載並啟動 OpenBnB MCP 伺服器，Semantic Kernel 隨後會連接到該伺服器以獲取真實的 Airbnb 數據。  


## 使用 OpenBnB MCP 伺服器運行代理

現在我們將運行連接到 OpenBnB MCP 伺服器的 AI 代理，搜尋斯德哥爾摩適合 2 位成人和 1 位小孩的真實 Airbnb 住宿。您可以隨意修改 `user_inputs` 列表來調整搜尋條件。


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

摘要  
恭喜你！你已成功建立了一個整合真實世界住宿搜尋功能的 AI 代理，並使用了模型上下文協議（MCP）：

使用的技術：  
- Semantic Kernel - 用於使用 Azure OpenAI 構建智能代理  
- Azure AI Foundry - 提供 LLM 功能和聊天完成  
- MCP（模型上下文協議）- 用於標準化工具整合  
- OpenBnB MCP Server - 提供真實的 Airbnb 搜尋功能  
- Node.js/NPX - 用於運行外部 MCP 伺服器  

你學到了什麼：  
- MCP 整合：將 Semantic Kernel 代理連接到外部 MCP 伺服器  
- 實時數據訪問：通過即時 API 搜尋實際的 Airbnb 物業  
- 協議通信：使用標準輸入輸出（stdio）在代理和 MCP 伺服器之間進行通信  
- 功能發現：自動發現 MCP 伺服器提供的可用功能  
- 流式響應：實時捕獲並記錄功能調用  
- HTML 渲染：以樣式化表格和互動顯示格式化代理響應  

下一步：  
- 整合更多 MCP 伺服器（如天氣、航班、餐廳）  
- 構建結合 MCP 和 A2A 協議的多代理系統  
- 為自己的數據來源創建自定義 MCP 伺服器  
- 實現跨會話的持久對話記憶  
- 將代理部署到 Azure Functions，並進行 MCP 伺服器協調  
- 添加用戶身份驗證和預訂功能  

MCP 架構的主要優勢：  
- 標準化：為 AI 代理連接外部工具提供通用協議  
- 實時數據：訪問來自各種服務的即時、最新信息  
- 可擴展性：輕鬆整合新的數據來源和工具  
- 互操作性：可跨不同的 AI 框架和代理平台運作  
- 職責分離：明確區分 AI 邏輯和外部數據訪問  



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。應以原始語言的文件作為權威來源。對於關鍵資訊，建議尋求專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解讀概不負責。
